In [1]:
import torch
from transformers import pipeline

#pipe = pipeline("text-generation", model="HuggingFaceH4/starchat-beta", torch_dtype=torch.bfloat16, device_map="auto")
pipe = pipeline("text-generation", model="/data/mistral/query-to-mql/exp-5/oct-28", torch_dtype=torch.float16, device_map="cpu")

OSError: /data/mistral/query-to-mql/exp-5/oct-28 does not appear to have a file named config.json. Checkout 'https://huggingface.co//data/mistral/query-to-mql/exp-5/oct-28/None' for available files.